In [2]:
import networkx as nx
from pathlib import Path
import yaml
from dataclasses import dataclass

In [3]:
@dataclass
class MachineNode:
    machine: str
    eut: int
    dur_ticks: int

@dataclass
class IngredientNode:
    name: str
    base_quant: int
    # Only used by Ingredients directly connected to machines
    base_direction: str
    associated_machine_index: int

@dataclass
class EdgeData:
    name: str
    base_quant: int

In [4]:
flow_projects_path = Path('~/Dropbox/OrderedSetCode/game-optimization/minecraft/flow/projects').expanduser()
light_fuel_path = flow_projects_path / 'power/oil/light_fuel.yaml'
with open(light_fuel_path, 'r') as f:
    conf = yaml.safe_load(f)
print(conf)

G = nx.MultiDiGraph()
# MultiDiGraph = DiGraph, but self edges and parallel edges are allowed
node_id = 0
edge_id = 0
for machine_dict in conf:
    # Add machine nodes
    machine_node_id = node_id
    G.add_node(node_id, object=MachineNode(machine_dict['m'], machine_dict['eut'], machine_dict['dur']))
    node_id += 1

    # Add ingredient nodes
    for direction in ['I', 'O']:
        for ingredient_name, ingredient_quantity in machine_dict[direction].items():
            G.add_node(node_id, object=IngredientNode(ingredient_name, ingredient_quantity, direction, machine_node_id))
            if direction == 'I':
                G.add_edge(machine_node_id, node_id, object=EdgeData(ingredient_name, ingredient_quantity))
            elif direction == 'O':
                G.add_edge(node_id, machine_node_id, object=EdgeData(ingredient_name, ingredient_quantity))
            node_id += 1
            edge_id += 1

print(G)
for x in range(G.number_of_nodes()):
    print(G.nodes[x])

[{'m': 'chemical reactor', 'tier': 'LV', 'I': {'hydrogen': 2000, 'sulfuric light fuel': 12000}, 'O': {'hydrogen sulfide': 1000, 'light fuel': 12000}, 'eut': 30, 'dur': 8, 'group': 'oil'}, {'m': 'distillery', 'tier': 'LV', 'I': {'oil': 25}, 'O': {'sulfuric light fuel': 25}, 'eut': 24, 'dur': 1, 'group': 'oil', 'number': 1}, {'m': 'electrolyzer', 'tier': 'LV', 'I': {'water': 3000}, 'O': {'hydrogen': 1000, 'oxygen': 500}, 'eut': 30, 'dur': 50, 'group': 'hydrogen'}]
MultiDiGraph with 12 nodes and 9 edges
{'object': MachineNode(machine='chemical reactor', eut=30, dur_ticks=8)}
{'object': IngredientNode(name='hydrogen', base_quant=2000, base_direction='I', associated_machine_index=0)}
{'object': IngredientNode(name='sulfuric light fuel', base_quant=12000, base_direction='I', associated_machine_index=0)}
{'object': IngredientNode(name='hydrogen sulfide', base_quant=1000, base_direction='O', associated_machine_index=0)}
{'object': IngredientNode(name='light fuel', base_quant=12000, base_direct

In [5]:
from collections import defaultdict

def produceConnectedGraphFromDisjoint(G: nx.MultiDiGraph) -> nx.MultiDiGraph:
    ingredient_by_type_to_node_id = {'I': defaultdict(list), 'O': defaultdict(list)}

    for idx, node in G.nodes.items():
        print(idx, node)
        if isinstance(node['object'], IngredientNode):
            ingredient_by_type_to_node_id[node['object'].base_direction][node['object'].name].append(idx)

    print(ingredient_by_type_to_node_id)

    return G

cg = produceConnectedGraphFromDisjoint(G)
print(cg)

0 {'object': MachineNode(machine='chemical reactor', eut=30, dur_ticks=8)}
1 {'object': IngredientNode(name='hydrogen', base_quant=2000, base_direction='I', associated_machine_index=0)}
2 {'object': IngredientNode(name='sulfuric light fuel', base_quant=12000, base_direction='I', associated_machine_index=0)}
3 {'object': IngredientNode(name='hydrogen sulfide', base_quant=1000, base_direction='O', associated_machine_index=0)}
4 {'object': IngredientNode(name='light fuel', base_quant=12000, base_direction='O', associated_machine_index=0)}
5 {'object': MachineNode(machine='distillery', eut=24, dur_ticks=1)}
6 {'object': IngredientNode(name='oil', base_quant=25, base_direction='I', associated_machine_index=5)}
7 {'object': IngredientNode(name='sulfuric light fuel', base_quant=25, base_direction='O', associated_machine_index=5)}
8 {'object': MachineNode(machine='electrolyzer', eut=30, dur_ticks=50)}
9 {'object': IngredientNode(name='water', base_quant=3000, base_direction='I', associated_mac

In [42]:
ag = nx.nx_agraph.to_agraph(G)
ag.draw("proto.pdf", prog='dot')